In [1]:
pip install karateclub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 873.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.8 MB/s eta 0:00:00
  Created wheel for karateclub: filename=karateclub-1.3.3-py3-none-any.whl size=101984 sha256=96ab81e2bf5b366b20240a03d858e8d077abf9d36ed49f3fd172ae2e18933e50
  Stored in directory: /root/.cache/pip/wheels/62/bd/af/17e7ca6ba0ed144d22502780f5c0660a8e4985939dc6973a81
Successfully built karateclub
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1

In [24]:
import math
import networkx as nx
from karateclub import Diff2Vec
import numpy as np
import pandas as pd
import random
# Define the community memberships for each node
# Assign community labels to nodes
np.random.seed(1)
n=2000
p0 = pd.read_csv("P_0_2000.csv")
p0=p0.values
p1 = pd.read_csv("P_1_2000.csv")
p1=p1.values
p2 = pd.read_csv("P_2_2000.csv")
p2=p2.values

p0_net = np.zeros_like(p0, dtype=int)

# Generate binary data based on the probabilities in the input matrix
for i in range(n):
    for j in range(i+1,n):
        p0_net[i][j] = np.random.choice([0, 1], p=[1 - p0[i][j], p0[i][j]])

for i in range(n):
    for j in range(i):
        p0_net[i][j] = p0_net[j][i]

p1_net = np.zeros_like(p1, dtype=int)

# Generate binary data based on the probabilities in the input matrix
for i in range(n):
    for j in range(i+1,n):
        p1_net[i][j] = np.random.choice([0, 1], p=[1 - p1[i][j], p1[i][j]])

for i in range(n):
    for j in range(i):
        p1_net[i][j] = p1_net[j][i]

p2_net = np.zeros_like(p2, dtype=int)

# Generate binary data based on the probabilities in the input matrix
for i in range(n):
    for j in range(i+1,n):
        p2_net[i][j] = np.random.choice([0, 1], p=[1 - p2[i][j], p2[i][j]])

for i in range(n):
    for j in range(i):
        p2_net[i][j] = p2_net[j][i]

rowsum=p0_net.sum(axis=1)
isoline=np.where(rowsum == 0)[0]
maxline=rowsum.argmax()
print(maxline)
print(isoline)
p0_net[isoline[0]][maxline]=1
p0_net[maxline][isoline[0]]=1
p0_net[isoline[1]][maxline]=1
p0_net[maxline][isoline[1]]=1

397
[1076 1136]


In [ ]:
B=100

G = nx.Graph(p0_net)
embedding_results = []
for _ in range(B):
    # Generate a stochastic block model graph based on the probability matrix
    # Perform node2vec embedding
    model=Diff2Vec(dimensions=3,diffusion_number=20,seed=_)
    model.fit(G)
    embeddings = model.get_embedding()
    embedding_results.append(embeddings)

merged_matrix_2logn = np.concatenate(embedding_results, axis=1)
df = pd.DataFrame(merged_matrix_2logn)
df.to_csv('20_Diff2Vec_2logn_embedding_results.csv', index=False)


G = nx.Graph(p1_net)
embedding_results = []
for _ in range(B):
    # Generate a stochastic block model graph based on the probability matrix
    # Perform node2vec embedding
    model=Diff2Vec(dimensions=3,diffusion_number=20,seed=_)
    model.fit(G)
    embeddings = model.get_embedding()
    embedding_results.append(embeddings)

merged_matrix_n12 = np.concatenate(embedding_results, axis=1)
df = pd.DataFrame(merged_matrix_n12)
df.to_csv('20_Diff2Vec_n12_embedding_results.csv', index=False)

G = nx.Graph(p2_net)
embedding_results = []
for _ in range(B):
    # Generate a stochastic block model graph based on the probability matrix
    # Perform node2vec embedding
    model=Diff2Vec(dimensions=3,diffusion_number=20,seed=_)
    model.fit(G)
    embeddings = model.get_embedding()
    embedding_results.append(embeddings)

merged_matrix_n23 = np.concatenate(embedding_results, axis=1)
df = pd.DataFrame(merged_matrix_n23)
df.to_csv('20_Diff2Vec_n23_embedding_results.csv', index=False)